In [1]:
import pandas as pd
from theano.sandbox import cuda
import utils; reload(utils)
from utils import *
from __future__ import division, print_function


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
import numpy as np

In [3]:
df_train = pd.read_csv('train.csv')
df_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0
5,82428052,Fried chickens \n\nIs dat sum fried chickens?,0,0,0,0,0,0
6,87311443,Why can you put English for example on some pl...,0,0,0,0,0,0
7,114749757,Guy Fawkes \n\nim a resident in bridgwater and...,0,0,0,0,0,0
8,138560519,as far as nicknames go this article is embarra...,0,0,0,0,0,0
9,139353149,Woodland Meadows\nGood to hear that you correc...,0,0,0,0,0,0


In [4]:
df_test = pd.read_csv('test.csv')
df_test

,id,comment_text
0,6044863,==Orphaned non-free media (Image:41cD1jboEvL. ...
1,6102620,::Kentuckiana is colloquial. Even though the ...
2,14563293,"Hello fellow Wikipedians,\nI have just modifie..."
3,21086297,"AKC Suspensions \nThe Morning Call - Feb 24, 2..."
4,22982444,== [WIKI_LINK: Talk:Celts] ==
5,24388733,Hello Xdes! [WIKI_LINK: Wikipedia:Introduction...
6,26195914,"Hi there, I'm . I just wanted to let you know ..."
7,31769073,::I don't think a stuffed arm really counts as...
8,35289443,==Was it ever really a single in the UK?==
9,38393350,Thanks for uploading [WIKI_LINK: Image:Cloudco...


In [5]:
df_x = df_train['comment_text']
df_y = df_train.drop('comment_text', axis=1)

In [6]:
df_y = df_y.drop('id', axis=1)
df_y

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0
7,0,0,0,0,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [7]:
X_train = df_x.as_matrix()
y_train = df_y.as_matrix()

In [15]:
X_train.shape

(95851,)

In [8]:
X_train

array(["Nonsense?  kiss off, geek. what I said is true.  I'll have your account terminated.",
       '"\n\n Please do not vandalize pages, as you did with this edit to W. S. Merwin. If you continue to do so, you will be blocked from editing.    "',
       '"\n\n ""Points of interest"" \n\nI removed the ""points of interest"" section you added because it seemed kind of spammy. I know you probably didn\'t mean to disobey the rules, but generally, a point of interest tends to be rather touristy, and quite irrelevant to an area culture. That\'s just my opinion, though.\n\nIf you want to reply, just put your reply here and add {{talkback|Jamiegraham08}} on my talkpage.   "',
       ...,
       'Mamoun Darkazanli\nFor some reason I am unable to fix the bold formatting on the Arabic name of Mamoun Darkazanli. The entire first paragraph is bolded and I just want to bold the script. Please take a look when you get the chance.',
       'Salafi would be a better term. It is more politically corre

In [14]:
vocab_list = []
for i in range(len(X_train)):
    if i%5000 == 0:
        print(i)
    temp_list = X_train[i].split(' ')
    for j in range(len(temp_list)):
        temp_list[j] = temp_list[j].lower()
        if temp_list[j] not in vocab_list:
            vocab_list.append(temp_list[j])

vocab_list

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000


['nonsense?',
 '',
 'kiss',
 'off,',
 'geek.',
 'what',
 'i',
 'said',
 'is',
 'true.',
 "i'll",
 'have',
 'your',
 'account',
 'terminated.',
 '"\n\n',
 'please',
 'do',
 'not',
 'vandalize',
 'pages,',
 'as',
 'you',
 'did',
 'with',
 'this',
 'edit',
 'to',
 'w.',
 's.',
 'merwin.',
 'if',
 'continue',
 'so,',
 'will',
 'be',
 'blocked',
 'from',
 'editing.',
 '"',
 '""points',
 'of',
 'interest""',
 '\n\ni',
 'removed',
 'the',
 'section',
 'added',
 'because',
 'it',
 'seemed',
 'kind',
 'spammy.',
 'know',
 'probably',
 "didn't",
 'mean',
 'disobey',
 'rules,',
 'but',
 'generally,',
 'a',
 'point',
 'interest',
 'tends',
 'rather',
 'touristy,',
 'and',
 'quite',
 'irrelevant',
 'an',
 'area',
 'culture.',
 "that's",
 'just',
 'my',
 'opinion,',
 'though.\n\nif',
 'want',
 'reply,',
 'put',
 'reply',
 'here',
 'add',
 '{{talkback|jamiegraham08}}',
 'on',
 'talkpage.',
 'asking',
 'some',
 'his',
 'nationality',
 'racial',
 'offence.',
 'wow',
 "wasn't",
 'aware',
 'it.',
 'block

In [23]:
random = X_train[1].split(' ')
random

['"\n\n',
 'Please',
 'do',
 'not',
 'vandalize',
 'pages,',
 'as',
 'you',
 'did',
 'with',
 'this',
 'edit',
 'to',
 'W.',
 'S.',
 'Merwin.',
 'If',
 'you',
 'continue',
 'to',
 'do',
 'so,',
 'you',
 'will',
 'be',
 'blocked',
 'from',
 'editing.',
 '',
 '',
 '',
 '"']

In [17]:
vocab_list[:10]

['revision?',
 'deserts',
 '""anarcho-syndicalism""',
 '(monetary',
 'available.""\nthe',
 'article???\ni',
 'approving',
 'bucharest',
 'obtunded.',
 'car!']

In [16]:
#shuffling vocab_list
from random import shuffle
vocabulary = shuffle(vocab_list)


In [18]:
vocab_list[:10]

['revision?',
 'deserts',
 '""anarcho-syndicalism""',
 '(monetary',
 'available.""\nthe',
 'article???\ni',
 'approving',
 'bucharest',
 'obtunded.',
 'car!']

In [47]:
test = X_train[0]
test

"Nonsense?  kiss off, geek. what I said is true.  I'll have your account terminated."

In [48]:
split = test.split(' ')
split

for i in range(len(split)):
    test = test.replace(split[i], str(i))

test


"10110010110 10110 10110k10110i10110s10110s10110 10110o10110f10110f10110,10110 10110g10110e10110e10110k10110.10110 10110w10110h10110a10110t10110 10110610110 10110s10110a10110i10110d10110 10110i10110s10110 10110t10110r10110u10110e10110.10110 10110 10110610110'10110l10110l10110 10110h10110a10110v10110e10110 10110y10110o10110u10110r10110 10110a10110c10110c10110o10110u10110n10110t10110 10110t10110e10110r10110m10110i10110n10110a10110t10110e10110d10110.10110"

In [41]:
test = test.replace('Nonsense', 'text')
test

"text?  kiss off, geek. what I said is true.  I'll have your account terminated."